<a href="https://colab.research.google.com/github/Rogerio-mack/Machine-Learning-I/blob/main/ML4_Knn_ex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **4. Classificação: Knn**
---



Após fazer os exercícios deste laboratório responda ao **questionário correspondente da aula no Moodle**.



# Caso: **Predição de Diagnósticos a partir de Dados de Imagens**


https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)

Os dados acima são características computadas a partir de uma imagem digitalizada por *agulha fina* (PAAF) de uma massa mamária. Eles descrevem características dos núcleos celulares presentes na imagem e classificam os tumores como malignos ou benignos.

Os dados estão na URL: https://meusite.mackenzie.br/rogerio/TIC/breast-cancer-wisconsin.csv





In [2]:
#@markdown **Set Up Lab** (*run this before continue*)

import pandas                  as pd
import numpy                   as np
import matplotlib.pyplot       as plt
import warnings
import os
import seaborn as sns
warnings.filterwarnings("ignore")

breast = pd.read_csv('https://github.com/Rogerio-mack/Machine-Learning-I/raw/main/data/breast.csv')

from sklearn.model_selection import train_test_split

seed = 1984
X_train, X_test, y_train, y_test = train_test_split(breast.drop(columns=['diagnosis']), breast.diagnosis, test_size=0.05, stratify=breast.diagnosis, random_state=seed)

breast = pd.concat([X_train, y_train],axis=1)
new_breast = X_test

breast.to_csv('breast.csv',index=False)

new_breast['diagnosis'] = '?'
new_breast.to_csv('new_breast.csv',index=False)

y_test.to_csv('new_breast_answers.csv',index=False)


E você empregará os dados já tratados no set up acima, com os dataframes `breast` e `new_breast`. A idea aqui é empregar os dados de `breast` para estimar as classes dos novos casos desconhecidos de `new_breast`.



In [3]:
breast.head()

,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,diagnosis
411,8911670,18.810,19.98,120.90,1102.0,0.08923,0.05884,0.08020,0.05843,0.1550,...,24.30,129.00,1236.0,0.12430,0.1160,0.2210,0.12940,0.2567,0.05737,M
149,9047,12.940,16.17,83.18,507.6,0.09879,0.08836,0.03296,0.02390,0.1735,...,23.02,89.69,580.9,0.11720,0.1958,0.1810,0.08388,0.3297,0.07834,B
103,859196,9.173,13.86,59.20,260.9,0.07721,0.08751,0.05988,0.02180,0.2341,...,19.23,65.59,310.1,0.09836,0.1678,0.1397,0.05087,0.3282,0.08490,B
260,864496,8.726,15.83,55.84,230.9,0.11500,0.08201,0.04132,0.01924,0.1649,...,19.62,64.48,284.4,0.17240,0.2364,0.2456,0.10500,0.2926,0.10170,B
106,91505,12.540,16.32,81.25,476.3,0.11580,0.10850,0.05928,0.03279,0.1943,...,21.40,86.67,552.0,0.15800,0.1751,0.1889,0.08411,0.3155,0.07538,B


In [4]:
new_breast.head()

,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,diagnosis
274,862009,13.45,18.30,86.60,555.1,0.10220,0.08165,0.03974,0.02780,0.1638,...,25.94,97.59,699.4,0.13390,0.17510,0.13810,0.07911,0.2678,0.06603,?
107,9110127,18.03,16.85,117.50,990.0,0.08947,0.12320,0.10900,0.06254,0.1720,...,22.02,133.30,1292.0,0.12630,0.26660,0.42900,0.15350,0.2842,0.08225,?
264,90769601,11.13,16.62,70.47,381.1,0.08151,0.03834,0.01369,0.01370,0.1511,...,20.29,74.35,421.1,0.10300,0.06219,0.04580,0.04044,0.2383,0.07083,?
175,909777,10.57,18.32,66.82,340.9,0.08142,0.04462,0.01993,0.01111,0.2372,...,23.31,69.35,366.3,0.09794,0.06542,0.03986,0.02222,0.2699,0.06736,?
215,904689,12.96,18.29,84.18,525.2,0.07351,0.07899,0.04057,0.01883,0.1874,...,24.61,96.31,621.9,0.09329,0.23180,0.16040,0.06608,0.3207,0.07247,?


# Exercício. Acesse e Explore os dados

Qual o atributo classe? Quantos casos são **'B'enignos** e **'M'alignos**? Existem dados ausentes (Null ou NA)?


In [5]:
target_attribute = breast.columns[-1]

num_benign_cases = breast[target_attribute].value_counts()['B']
num_malignant_cases = breast[target_attribute].value_counts()['M']

missing_data = breast.isnull().sum().any()

print("Target attribute:", target_attribute)
print("Number of benign cases (B):", num_benign_cases)
print("Number of malignant cases (M):", num_malignant_cases)
print("Are there any missing data (Null or NA)?:", missing_data)


Target attribute: diagnosis
Number of benign cases (B): 322
Number of malignant cases (M): 191
Are there any missing data (Null or NA)?: False


# Exercício. Preparando os dados

Prepare aqui os dados de entrada `X` e saída `y` do modelo.

Note que há um atributo além do atributo classe que não é preditor (qual?). Remova esse atributo antes de normalizar os dados. Para normalização empregue o `MinMaxScaler()` do scikit-learn (ver no material de teoria exemplo de como aplicar).

In [6]:
# Seu código aqui
from sklearn.preprocessing import MinMaxScaler

features = breast.drop(columns=[target_attribute])

scaler = MinMaxScaler()
normalized_features = scaler.fit_transform(features)

labels = breast[target_attribute]



# Exercício. Conjunto de Treinamento e Teste

Empregue a função `train_test_split` e separe 0.3 dos dados para teste. Empregue o `random_state=123` para garantir a reprodutibilidade dos resultados e empregue o parâmetro `stratify` para garantir a mesma distribuição das classes nos conjuntos de treinamento e teste.

In [7]:
# Seu código aqui
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(normalized_features, labels, test_size=0.25, random_state=42, stratify=labels)


# Exercício. Treine seu modelo Knn

Treine dois modelos Knn, um com k=3 e outro com k=11. Verifique a acuracidade de cada modelo para o conjunto de teste.





In [15]:
# Seu código aqui
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

knn_model_3 = KNeighborsClassifier(n_neighbors=5)
knn_model_3.fit(X_train, y_train)

y_predicted = knn_model_3.predict(X_test)

accuracy_3 = accuracy_score(y_test, y_predicted)
print("Accuracy of KNN model with k=3:", accuracy_3)

Accuracy of KNN model with k=3: 0.9767441860465116


In [14]:
# Seu código aqui
from sklearn.neighbors import KNeighborsClassifier
knn_model_11 = KNeighborsClassifier(n_neighbors=11)
knn_model_11.fit(X_train, y_train)

y_predicted_11 = knn_model_11.predict(X_test)

accuracy_11 = accuracy_score(y_test, y_predicted_11)
print("Accuracy of KNN model with k=11:", accuracy_11)


Accuracy of KNN model with k=11: 0.9844961240310077


# Exercício. Classification Report

Produza o Classification report para o modelo com maior acuracidade obtido acima.

In [16]:
# Seu código aqui
from sklearn.metrics import classification_report
if accuracy_3 > accuracy_11:
    chosen_model = knn_model_3
else:
    chosen_model = knn_model_11

y_predicted_final = chosen_model.predict(X_test)

classification_report_final = classification_report(y_test, y_predicted_final)

print("Final Classification Report:")
print(classification_report_final)



Final Classification Report:
              precision    recall  f1-score   support

           B       0.98      1.00      0.99        81
           M       1.00      0.96      0.98        48

    accuracy                           0.98       129
   macro avg       0.99      0.98      0.98       129
weighted avg       0.98      0.98      0.98       129



# Exercício. Prevendo novos Casos

Empregue o melhor modelo obtido acima para classificar os novos casos de `new_breast`. Quantos casos benignos e malignos você encontra?

**Dica**: verifique que os dados dos novos casos precisam estar no mesmo *formato* dos dados empregados no treinamento. Verifique por exemplo os valores de X_train, X_test e compare com os dados de `new_breast`. Veja que os dados parecem estar muito diferentes agora, por que?

In [17]:
# Seu código aqui
new_breast_normalized = scaler.transform(new_breast.drop(columns=['diagnosis']))

y_predicted_new = chosen_model.predict(new_breast_normalized)

num_benign_new = np.sum(y_predicted_new == 'B')
num_malignant_new = np.sum(y_predicted_new == 'M')

print("Number of benign cases in new_breast:", num_benign_new)
print("Number of malignant cases in new_breast:", num_malignant_new)


Number of benign cases in new_breast: 17
Number of malignant cases in new_breast: 10
